# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,palora,-1.7000,-77.9333,29.94,40,8,2.43,EC,1723145323
1,1,port-aux-francais,-49.3500,70.2167,1.57,74,98,9.70,TF,1723145326
2,2,ushuaia,-54.8000,-68.3000,5.81,75,20,0.51,AR,1723145327
3,3,grytviken,-54.2811,-36.5092,-11.14,74,17,6.58,GS,1723145328
4,4,port elizabeth,-33.9180,25.5701,14.62,90,0,3.60,ZA,1723145329


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:

%%capture --no-display
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    tiles='OSM', 
    color='City', 
    size=10, 
    hover_cols=['City', 'Humidity']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_city_data_df = city_data_df[(city_data_df['Max Temp'] <= 25) & (city_data_df['Max Temp'] >= 20) & (city_data_df['Humidity'] < 80)]

# Drop any rows with null values
filtered_city_data_df = filtered_city_data_df.dropna()

# Display sample data
filtered_city_data_df.head(25)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,adamstown,-25.0660,-130.1015,21.63,78,76,10.04,PN,1723145331
12,12,nuku'alofa,-21.1333,-175.2000,20.19,73,75,3.60,TO,1723145337
24,24,yellowknife,62.4560,-114.3525,20.79,57,20,8.23,CA,1723145350
26,26,fomboni,-12.2800,43.7425,24.99,71,0,1.81,KM,1723145352
36,36,port saint john's,-31.6229,29.5448,23.34,37,0,2.26,ZA,1723145363
44,44,whitehorse,60.7161,-135.0538,22.43,46,0,5.66,CA,1723145371
47,47,korla,41.7597,86.1469,24.15,40,0,2.13,CN,1723145246
54,54,lompoc,34.6391,-120.4579,21.40,68,0,7.20,US,1723145383
76,76,beau vallon,-20.4189,57.6953,22.13,73,40,5.14,MU,1723145408
89,89,saynshand,44.8824,110.1163,24.67,46,100,4.71,MN,1723145423


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head(25)

,City,Country,Lat,Lng,Humidity,Hotel Name
6,adamstown,PN,-25.0660,-130.1015,78,
12,nuku'alofa,TO,-21.1333,-175.2000,73,
24,yellowknife,CA,62.4560,-114.3525,57,
26,fomboni,KM,-12.2800,43.7425,71,
36,port saint john's,ZA,-31.6229,29.5448,37,
44,whitehorse,CA,60.7161,-135.0538,46,
47,korla,CN,41.7597,86.1469,40,
54,lompoc,US,34.6391,-120.4579,68,
76,beau vallon,MU,-20.4189,57.6953,73,
89,saynshand,MN,44.8824,110.1163,46,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000  # Radius in meters (10 km)
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
nuku'alofa - nearest hotel: City Hotel
yellowknife - nearest hotel: No hotel found
fomboni - nearest hotel: Mohéli La Plage
port saint john's - nearest hotel: Outback Inn
whitehorse - nearest hotel: Town & Mountain Hotel
korla - nearest hotel: Silver Star Hotel
lompoc - nearest hotel: Red Roof Inn Lompoc
beau vallon - nearest hotel: Le Jardin de Beau Vallon
saynshand - nearest hotel: No hotel found
colonia - nearest hotel: Wasserturm Hotel Cologne
mount pearl - nearest hotel: Sandman Signature St. John's Hotel
port mathurin - nearest hotel: Escale Vacances
hami - nearest hotel: Хами
avarua - nearest hotel: Paradise Inn
conception bay south - nearest hotel: No hotel found
georgievka - nearest hotel: No hotel found
peace river - nearest hotel: New Ridge Inn
pontevico - nearest hotel: No hotel found
st. john's - nearest hotel: Alt Hotel
glace bay - nearest hotel: No hotel found
santa ana de yacuma - nearest hotel: No hotel fo

,City,Country,Lat,Lng,Humidity,Hotel Name
6,adamstown,PN,-25.0660,-130.1015,78,No hotel found
12,nuku'alofa,TO,-21.1333,-175.2000,73,City Hotel
24,yellowknife,CA,62.4560,-114.3525,57,No hotel found
26,fomboni,KM,-12.2800,43.7425,71,Mohéli La Plage
36,port saint john's,ZA,-31.6229,29.5448,37,Outback Inn
...,...,...,...,...,...,...
547,maputo,MZ,-25.9653,32.5892,44,Mozaika
548,drayton valley,CA,53.2168,-114.9854,49,Lakeview Inn & Suites
553,ayacucho,PE,-13.1583,-74.2239,20,Hotel Las Americas
554,harper,LR,4.3750,-7.7169,78,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    'Lng', 'Lat', geo=True, tiles='OSM', size='Humidity',
    color='City', hover_cols=['City', 'Country', 'Hotel Name', 'Humidity']
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)